In [1]:
import pandas as pd
import numpy as np
import torch
from torcheval.metrics import R2Score
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor


/home/huopuz/iTransformer-fund_perf/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("dataset/hf/hf.csv")

perm_cols = ['date', 'PRODUCTREFERENCE']
add_cols = [ 
            # 'ret',
        'aum', 
        'SENT_', 
        # 'PTFSBD', 
        # 'PTFSFX', 
        # 'PTFSCOM', 
        # 'em', 
        # 'sp500', 
        # 'sizespread', 
        # 'bondmkt', 
        # 'creditspread',
        'SMB', 'HML','RF', 'mom', 
        # 'con','ipg','tfp','term','def','dei','mkt','lab',
        'confeature', 'tfpfeature', 'ipgfeature', 'termfeature', 'deffeature', 'deifeature', 'mktfeature', 'labfeature',
        # 'ret',
        'exret',
        ]

df = df[perm_cols + add_cols]

In [3]:
seq_len = 12
pred_len = 3
window_size = seq_len + pred_len

In [4]:
dataset = df.copy()
dataset = dataset.dropna()
train_df = dataset[dataset.date <= '2023-09-15']
test_df = dataset[dataset.date >= '2023-10-15']

In [5]:
test_fund_df = test_df.groupby('PRODUCTREFERENCE').agg({'date': 'count'}).reset_index()
test_df = test_df[test_df.PRODUCTREFERENCE.isin(list(test_fund_df[test_fund_df.date >= 3]['PRODUCTREFERENCE']))]

In [6]:
predict_df = dataset[(dataset.date >= '1995-10-15') & ( dataset.date <= '2023-09-15')]
# count_df = predict_df.groupby(['PRODUCTREFERENCE']).agg({'date': 'count'}).reset_index()
# predict_df.merge(count_df.loc[count_df.date >= 12], on='PRODUCTREFERENCE')
funds_to_eval = list(predict_df.merge(test_df, on='PRODUCTREFERENCE', how='inner')['PRODUCTREFERENCE'].unique())
predict_df = predict_df[predict_df.PRODUCTREFERENCE.isin(funds_to_eval)]
predict_df['series_id'] = predict_df['PRODUCTREFERENCE']

# if only including the products with enough history
# predict_fund_history_count = predict_df.groupby(['PRODUCTREFERENCE']).agg({'date':'count'}).reset_index()
# predict_funds = list(predict_fund_history_count.loc[predict_fund_history_count.date >= seq_len, 'PRODUCTREFERENCE'])
# predict_df = predict_df[predict_df.PRODUCTREFERENCE.isin(predict_funds)]
predict_df 

,date,PRODUCTREFERENCE,aum,SENT_,SMB,HML,RF,mom,confeature,tfpfeature,ipgfeature,termfeature,deffeature,deifeature,mktfeature,labfeature,exret,series_id
692,2003-03-15,29,2.058009e+07,-0.348188,0.0065,-0.0193,0.0010,0.0148,0.005962,0.012619,-0.011212,-0.000027,0.001656,-0.002744,0.007265,0.029133,0.001300,29
693,2003-04-15,29,2.168269e+07,-0.656042,0.0102,0.0114,0.0010,-0.0933,0.007559,0.025769,0.000326,-0.000203,-0.001717,0.001731,0.052591,0.010757,0.089247,29
694,2003-05-15,29,2.274405e+07,-0.666281,0.0481,0.0040,0.0009,-0.1070,0.004373,0.033915,0.001699,0.001653,-0.001125,0.006425,0.041822,0.047230,0.058401,29
695,2003-06-15,29,2.426419e+07,-0.587053,0.0166,0.0011,0.0010,-0.0100,0.011920,0.037656,0.004866,0.000292,-0.000927,-0.001493,0.010189,0.029317,0.010773,29
696,2003-07-15,29,2.520923e+07,-0.789554,0.0454,-0.0124,0.0007,-0.0024,0.008476,0.045766,-0.001639,-0.013977,0.001261,0.005395,0.016750,0.017129,0.013181,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433584,2023-05-15,106517,1.680000e+07,-0.238204,-0.0038,-0.0772,0.0036,-0.0063,0.000000,0.001681,-0.001138,0.000376,-0.004719,-0.002436,0.000357,0.002889,-0.056724,106517
433585,2023-06-15,106517,1.648000e+07,-0.226543,0.0134,-0.0026,0.0040,-0.0237,0.003690,-0.001614,-0.003789,-0.002197,-0.001130,0.018352,0.006218,0.008152,0.020602,106517
433586,2023-07-15,106517,1.633000e+07,-0.037569,0.0286,0.0411,0.0045,-0.0398,0.025138,0.000237,0.005394,-0.004979,0.003786,-0.012978,0.004173,-0.008211,0.030605,106517
433587,2023-08-15,106517,1.653804e+07,-0.041535,-0.0365,-0.0106,0.0045,0.0377,0.000000,0.001255,-0.000804,-0.005218,0.002439,-0.001877,-0.003547,-0.018627,0.032693,106517


In [7]:
def create_sliding_windows(data, window_size):
    windows = []
    
    if len(data) < window_size:
        window = data.copy()
        window.reset_index(drop=True, inplace=True)
        windows.append(window)
    else:
        for i in range(len(data) - window_size + 1):
            window = data.iloc[i:i+window_size].copy()
            window.reset_index(drop=True, inplace=True)
            windows.append(window)
    return pd.concat(windows, keys=range(len(windows)))


In [8]:
windows = []

for i in train_df.PRODUCTREFERENCE.unique():
    temp_df = train_df[train_df.PRODUCTREFERENCE == i]
    if len(temp_df) < 8:
        continue

    sliding_windows = create_sliding_windows(temp_df, window_size)
    sliding_windows = sliding_windows.reset_index()
    sliding_windows['series_id'] = sliding_windows['PRODUCTREFERENCE'].astype("string")  + '_' +  \
                                   sliding_windows['level_0'].astype("string")
    sliding_windows = sliding_windows.drop(columns=['level_0', 'level_1'])
    windows.append(sliding_windows)



In [23]:
train_data_df = pd.concat(windows).reset_index(drop=True)
train_data_df = train_data_df.rename(columns={'PRODUCTREFERENCE':'fund_id'})

In [25]:
cols = list(train_data_df.columns)
train_data_df = train_data_df[cols[:2] + [cols[-1]] + cols[2:-1]]
train_data_df.head(32)

,date,fund_id,series_id,aum,SENT_,SMB,HML,RF,mom,confeature,tfpfeature,ipgfeature,termfeature,deffeature,deifeature,mktfeature,labfeature,exret
0,2003-03-15,1,1_0,68531426.00,-0.348188,0.0065,-0.0193,0.0010,0.0148,8.464196e-04,-0.000100,0.002948,-0.000016,0.000989,-0.000768,0.000187,0.008730,0.0099
1,2003-04-15,1,1_0,70000000.00,-0.656042,0.0102,0.0114,0.0010,-0.0933,1.121613e-03,-0.000133,-0.000119,-0.000117,-0.001021,0.000363,0.001502,0.003320,0.0112
2,2003-05-15,1,1_0,75718485.00,-0.666281,0.0481,0.0040,0.0009,-0.1070,5.669380e-04,0.000422,-0.000724,0.000623,-0.000983,0.000595,0.001411,0.013864,0.0125
3,2003-06-15,1,1_0,73000000.00,-0.587053,0.0166,0.0011,0.0010,-0.0100,1.538602e-03,0.000990,-0.001858,0.000044,-0.001130,-0.000145,0.000403,0.006868,-0.0057
4,2003-07-15,1,1_0,72801540.49,-0.789554,0.0454,-0.0124,0.0007,-0.0024,8.006842e-04,-0.000081,0.000713,-0.001630,0.001840,0.000574,0.000580,0.003495,-0.0109
5,2003-08-15,1,1_0,71400000.00,-0.896371,0.0248,0.0153,0.0007,-0.0062,7.442238e-04,-0.002335,-0.002363,-0.002597,0.000000,-0.000326,0.000419,0.005119,-0.0161
6,2003-09-15,1,1_0,72655977.00,-0.750072,0.0054,0.0016,0.0008,-0.0019,-9.324400e-05,-0.004726,-0.000830,0.001407,0.000795,-0.000693,-0.000125,0.003030,0.0139
7,2003-10-15,1,1_0,75170000.00,-0.683628,0.0271,0.0197,0.0007,0.0399,-6.099880e-04,-0.001673,-0.005159,0.000166,0.000599,0.000647,0.000275,0.004021,0.0166
8,2003-11-15,1,1_0,76100000.00,-0.586979,0.0233,0.0178,0.0007,0.0146,-2.664121e-03,-0.001820,0.000354,0.002523,0.000334,0.001166,0.000191,0.006200,0.0096
9,2003-12-15,1,1_0,76200000.00,-0.755119,-0.0249,0.0160,0.0008,-0.0557,-7.220000e-04,-0.001960,-0.001335,0.000593,0.000520,0.000396,0.000747,0.002508,0.0035


In [38]:
train_data_df['date'] = pd.to_datetime(train_data_df['date'], format= '%Y-%m-%d' )
train_data_df

,date,PRODUCTREFERENCE,aum,SENT_,SMB,HML,RF,mom,confeature,tfpfeature,ipgfeature,termfeature,deffeature,deifeature,mktfeature,labfeature,exret,series_id
0,1997-04-15,21,53831320.0,0.947615,-0.0570,0.0007,0.0043,0.0484,0.004313,0.026873,0.000075,-0.000511,-7.074096e-03,-0.000491,0.023940,0.001495,-0.0111,21_0
1,1997-05-15,21,59350872.0,0.805671,0.0475,-0.0413,0.0049,-0.0517,0.001018,0.017150,-0.000125,-0.000956,3.815385e-03,0.000929,0.035814,0.009077,0.0764,21_0
2,1997-06-15,21,62195889.0,0.720814,0.0119,0.0158,0.0037,0.0260,0.001559,0.029962,-0.000123,0.000000,-4.613490e-03,-0.010451,0.025901,0.009209,0.0341,21_0
3,1997-07-15,21,68154647.0,0.517041,-0.0276,0.0026,0.0043,0.0377,0.006421,0.026881,0.001176,0.000167,3.685203e-16,0.000035,0.046172,0.009455,0.0399,21_0
4,1997-08-15,21,69194233.0,0.641085,0.0761,0.0118,0.0041,-0.0256,0.001608,0.028533,0.003461,-0.000964,-4.248750e-03,-0.000551,-0.027002,0.013776,0.0068,21_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4641214,2023-05-15,104220,15551300.0,-0.238204,-0.0038,-0.0772,0.0036,-0.0063,0.000000,0.000478,-0.002119,-0.008066,4.581084e-03,-0.002779,0.001277,-0.004740,-0.0188,104220_9
4641215,2023-06-15,104220,16250000.0,-0.226543,0.0134,-0.0026,0.0040,-0.0237,0.001918,-0.001410,-0.005605,-0.010420,1.260502e-03,0.019438,0.023334,-0.005216,0.0399,104220_9
4641216,2023-07-15,104220,16700000.0,-0.037569,0.0286,0.0411,0.0045,-0.0398,0.003145,-0.000367,0.005036,-0.006724,-3.717441e-03,-0.017034,0.011628,-0.016692,0.0201,104220_9
4641217,2023-08-15,104220,16645800.0,-0.041535,-0.0365,-0.0106,0.0045,0.0377,0.000000,0.000344,-0.000662,-0.006701,-2.200431e-03,-0.003180,-0.008859,-0.026102,0.0005,104220_9


In [24]:
static_features_df = train_data_df[['series_id', 'PRODUCTREFERENCE']].drop_duplicates().reset_index(drop=True)
static_features_df

,series_id,PRODUCTREFERENCE
0,1_0,1
1,1_1,1
2,1_2,1
3,1_3,1
4,1_4,1
...,...,...
145668,106523_20,106523
145669,106523_21,106523
145670,106523_22,106523
145671,106523_23,106523


In [26]:
predict_static_features = predict_df[['series_id', 'PRODUCTREFERENCE']].drop_duplicates()
predict_static_features

,series_id,PRODUCTREFERENCE
879,29,29
1419,35,35
7869,441,441
10655,727,727
11702,814,814
...,...,...
432661,105997,105997
432998,106455,106455
433242,106484,106484
433310,106485,106485


In [27]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_data_df.drop(columns=['PRODUCTREFERENCE']),
    id_column="series_id",
    timestamp_column="date", 
    static_features_df=static_features_df,
)
train_data.head()


aum     SENT_     SMB     HML      RF     mom  \
item_id timestamp                                                           
1_0     2003-03-15  68531426.00 -0.348188  0.0065 -0.0193  0.0010  0.0148   
        2003-04-15  70000000.00 -0.656042  0.0102  0.0114  0.0010 -0.0933   
        2003-05-15  75718485.00 -0.666281  0.0481  0.0040  0.0009 -0.1070   
        2003-06-15  73000000.00 -0.587053  0.0166  0.0011  0.0010 -0.0100   
        2003-07-15  72801540.49 -0.789554  0.0454 -0.0124  0.0007 -0.0024   

                    confeature  tfpfeature  ipgfeature  termfeature  \
item_id timestamp                                                     
1_0     2003-03-15    0.000846   -0.000100    0.002948    -0.000016   
        2003-04-15    0.001122   -0.000133   -0.000119    -0.000117   
        2003-05-15    0.000567    0.000422   -0.000724     0.000623   
        2003-06-15    0.001539    0.000990   -0.001858     0.000044   
        2003-07-15    0.000801   -0.000081    0.000713    -0.001630   

                    deffeature  deifeature  mktfeature  labfeature   exret  
item_id timestamp                                                           
1_0     2003-03-15    0.000989   -0.000768    0.000187    0.008730  0.0099  
        2003-04-15   -0.001021    0.000363    0.001502    0.003320  0.0112  
        2003-05-15   -0.000983    0.000595    0.001411    0.013864  0.0125  
        2003-06-15   -0.001130   -0.000145    0.000403    0.006868 -0.0057  
        2003-07-15    0.001840    0.000574    0.000580    0.003495 -0.0109

In [16]:
predict_data = TimeSeriesDataFrame.from_data_frame(
    predict_df.drop(columns=['PRODUCTREFERENCE']),
    id_column="series_id",
    timestamp_column="date", 
    static_features_df=predict_static_features,
)

In [17]:
predict_data

aum     SENT_     SMB     HML      RF     mom  \
item_id timestamp                                                            
29      2018-10-15  1.321388e+07 -0.337726 -0.0446  0.0340  0.0019 -0.0199   
        2018-11-15  1.347676e+07 -0.376686 -0.0077  0.0030  0.0018 -0.0137   
        2018-12-15  1.257233e+07 -0.423311 -0.0288 -0.0184  0.0020  0.0211   
        2019-01-15  1.347941e+07 -0.253061  0.0303 -0.0046  0.0021 -0.0868   
        2019-02-15  1.755835e+07 -0.210551  0.0174 -0.0267  0.0018  0.0086   
...                          ...       ...     ...     ...     ...     ...   
106517  2023-05-15  1.680000e+07 -0.238204 -0.0038 -0.0772  0.0036 -0.0063   
        2023-06-15  1.648000e+07 -0.226543  0.0134 -0.0026  0.0040 -0.0237   
        2023-07-15  1.633000e+07 -0.037569  0.0286  0.0411  0.0045 -0.0398   
        2023-08-15  1.653804e+07 -0.041535 -0.0365 -0.0106  0.0045  0.0377   
        2023-09-15  1.618329e+07 -0.036615 -0.0180  0.0152  0.0043  0.0026   

                    confeature  tfpfeature  ipgfeature  termfeature  \
item_id timestamp                                                     
29      2018-10-15    0.000851   -0.000436    0.003126     0.001544   
        2018-11-15   -0.003768    0.000402    0.000014     0.000110   
        2018-12-15    0.004350   -0.000332    0.000248     0.012452   
        2019-01-15    0.038914    0.004579    0.000160     0.000437   
        2019-02-15   -0.008847    0.048079    0.004298     0.021814   
...                        ...         ...         ...          ...   
106517  2023-05-15    0.000000    0.001681   -0.001138     0.000376   
        2023-06-15    0.003690   -0.001614   -0.003789    -0.002197   
        2023-07-15    0.025138    0.000237    0.005394    -0.004979   
        2023-08-15    0.000000    0.001255   -0.000804    -0.005218   
        2023-09-15    0.017384    0.001656    0.001839    -0.004237   

                    deffeature  deifeature  mktfeature  labfeature     exret  
item_id timestamp                                                             
29      2018-10-15   -0.002889   -0.003736   -0.032014    0.008974 -0.054000  
        2018-11-15   -0.011617    0.014797    0.008539    0.003616  0.017000  
        2018-12-15   -0.010352    0.006612   -0.047732    0.007304 -0.069000  
        2019-01-15   -0.021769    0.001113    0.046051   -0.005369  0.069800  
        2019-02-15    0.004729   -0.002780    0.020619    0.016597  0.012500  
...                        ...         ...         ...         ...       ...  
106517  2023-05-15   -0.004719   -0.002436    0.000357    0.002889 -0.056724  
        2023-06-15   -0.001130    0.018352    0.006218    0.008152  0.020602  
        2023-07-15    0.003786   -0.012978    0.004173   -0.008211  0.030605  
        2023-08-15    0.002439   -0.001877   -0.003547   -0.018627  0.032693  
        2023-09-15    0.007439   -0.002021   -0.007604   -0.022687  0.008251  

[16004 rows x 15 columns]

In [18]:
# hyperparameters = {
#     'TemporalFusionTransformer': {
#         'context_length': 12  # Fixed look-back window of 5 periods
#     },
#     'DeepAR': {
#         'context_length': 12  # Fixed look-back window of 5 periods
#     },
#     # 'RecursiveTabular': {
#     #     'lags': 12
#     # },
#     # 'DirectTabular': {
#     #     'lags': 12
#     # },
    
# }

hyperparameters = {
    'TemporalFusionTransformer': {
        # 'context_length': 36  # Fixed look-back window of 5 periods
    },
    # 'DirectTabular':{},
    # 'Theta':{},
    # 'ETS':{},


    
}

In [28]:
predictor = TimeSeriesPredictor(
    prediction_length=3,
    path="benchmark/autogluon-48-3-monthly",
    target="exret",
    eval_metric="MASE",
    freq="MS",
    # verbosity=4,
)


predictor.fit(
    train_data=train_data,
    hyperparameters=hyperparameters,
    # presets="medium_quality",

    
)

Beginning AutoGluon training...
AutoGluon will save models to 'benchmark/autogluon-48-3-monthly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          32
GPU Count:          1
Memory Avail:       8.91 GB / 15.48 GB (57.6%)
Disk Space Avail:   69.99 GB / 250.00 GB (28.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'MS',
 'hyperparameters': {'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 3,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'exret',
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'MS'.
Provided train_data has 5890026 rows (NaN fra

In [29]:
predict_data

aum     SENT_     SMB     HML      RF     mom  \
item_id timestamp                                                            
29      2018-10-15  1.321388e+07 -0.337726 -0.0446  0.0340  0.0019 -0.0199   
        2018-11-15  1.347676e+07 -0.376686 -0.0077  0.0030  0.0018 -0.0137   
        2018-12-15  1.257233e+07 -0.423311 -0.0288 -0.0184  0.0020  0.0211   
        2019-01-15  1.347941e+07 -0.253061  0.0303 -0.0046  0.0021 -0.0868   
        2019-02-15  1.755835e+07 -0.210551  0.0174 -0.0267  0.0018  0.0086   
...                          ...       ...     ...     ...     ...     ...   
106517  2023-05-15  1.680000e+07 -0.238204 -0.0038 -0.0772  0.0036 -0.0063   
        2023-06-15  1.648000e+07 -0.226543  0.0134 -0.0026  0.0040 -0.0237   
        2023-07-15  1.633000e+07 -0.037569  0.0286  0.0411  0.0045 -0.0398   
        2023-08-15  1.653804e+07 -0.041535 -0.0365 -0.0106  0.0045  0.0377   
        2023-09-15  1.618329e+07 -0.036615 -0.0180  0.0152  0.0043  0.0026   

                    confeature  tfpfeature  ipgfeature  termfeature  \
item_id timestamp                                                     
29      2018-10-15    0.000851   -0.000436    0.003126     0.001544   
        2018-11-15   -0.003768    0.000402    0.000014     0.000110   
        2018-12-15    0.004350   -0.000332    0.000248     0.012452   
        2019-01-15    0.038914    0.004579    0.000160     0.000437   
        2019-02-15   -0.008847    0.048079    0.004298     0.021814   
...                        ...         ...         ...          ...   
106517  2023-05-15    0.000000    0.001681   -0.001138     0.000376   
        2023-06-15    0.003690   -0.001614   -0.003789    -0.002197   
        2023-07-15    0.025138    0.000237    0.005394    -0.004979   
        2023-08-15    0.000000    0.001255   -0.000804    -0.005218   
        2023-09-15    0.017384    0.001656    0.001839    -0.004237   

                    deffeature  deifeature  mktfeature  labfeature     exret  
item_id timestamp                                                             
29      2018-10-15   -0.002889   -0.003736   -0.032014    0.008974 -0.054000  
        2018-11-15   -0.011617    0.014797    0.008539    0.003616  0.017000  
        2018-12-15   -0.010352    0.006612   -0.047732    0.007304 -0.069000  
        2019-01-15   -0.021769    0.001113    0.046051   -0.005369  0.069800  
        2019-02-15    0.004729   -0.002780    0.020619    0.016597  0.012500  
...                        ...         ...         ...         ...       ...  
106517  2023-05-15   -0.004719   -0.002436    0.000357    0.002889 -0.056724  
        2023-06-15   -0.001130    0.018352    0.006218    0.008152  0.020602  
        2023-07-15    0.003786   -0.012978    0.004173   -0.008211  0.030605  
        2023-08-15    0.002439   -0.001877   -0.003547   -0.018627  0.032693  
        2023-09-15    0.007439   -0.002021   -0.007604   -0.022687  0.008251  

[16004 rows x 15 columns]

In [30]:
predictions = predictor.predict(predict_data)
predictions.head()

data with frequency 'None' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: TemporalFusionTransformer


mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                      
29      2023-10-01  0.016359 -0.031850 -0.011720  0.000558  0.009764   
        2023-11-01  0.017193 -0.025018 -0.007596  0.003892  0.011263   
        2023-12-01  0.017027 -0.019589 -0.004618  0.005696  0.011875   
35      2023-10-01  0.009378 -0.013475 -0.003925  0.001651  0.006333   
        2023-11-01  0.010172 -0.011307 -0.002161  0.003550  0.007236   

                         0.5       0.6       0.7       0.8       0.9  
item_id timestamp                                                     
29      2023-10-01  0.016359  0.025081  0.036892  0.054590  0.075053  
        2023-11-01  0.017193  0.024977  0.035969  0.052554  0.071784  
        2023-12-01  0.017027  0.024467  0.034189  0.049050  0.067613  
35      2023-10-01  0.009378  0.013495  0.018476  0.026939  0.036781  
        2023-11-01  0.010172  0.013976  0.019346  0.027580  0.037814

In [31]:
pred_df = predictions.reset_index()
pred_df['yearmonth'] = pred_df.timestamp.dt.strftime('%Y-%m')
test_df['yearmonth'] = test_df.date.str.slice(0, -3)
eval_df = pred_df.merge(test_df, how="inner", left_on=['item_id', 'yearmonth'], right_on=['PRODUCTREFERENCE', 'yearmonth'])

In [32]:
preds = list(eval_df['mean'])
trues = list(eval_df['exret'])

metrics = R2Score()
# input = torch.tensor(preds[:,:,0].flatten())
# target = torch.tensor(trues[:,:,0].flatten())

input = torch.tensor(preds)
target = torch.tensor(trues)

metrics.update(input, target)
print(metrics.compute())

tensor(0.0314)


: 